In [1]:
from ftplib import FTP
from datetime import datetime
import os
import ftplib
import time
import re
import shutil

In [2]:
# Connect to the FTP server
ftp = FTP("120.28.19.206")

In [3]:
# Login to the FTP server
ftp.login(user="MORE_01", passwd="ar@pNkRW4v")

'230 User logged in.'

In [4]:
ftp.cwd('/LMP/DIPC/')

'250 CWD command successful.'

In [5]:
# Function to download files from the FTP server
def download_files():
    now = datetime.now()  # current date and time
    date_today = now.strftime("%Y%m%d")

    ftp_folder_path = "/LMP/DIPC/"
    local_folder_path = r"C:\Users\aslee\OneDrive - MORE ELECTRIC AND POWER CORPORATION\Desktop\DASHBOARD_FINAL\MORE Trading Node"

    # List the files in the FTP directory
    files = ftp.nlst()

    for addtl_folder in files:
        # if the filename is the same with the date today
        if addtl_folder == date_today:
            ftp_folder_file = f"/LMP/DIPC/{addtl_folder}"
            local_folder_file = os.path.join(local_folder_path, addtl_folder)

            print(f"Checking FTP folder path: {ftp_folder_file}")
            print(f"Checking local folder path: {local_folder_file}")

            # Check if the folder exists on the FTP server
            folder_exists = True
            try:
                ftp.cwd(ftp_folder_file)
            except ftplib.error_perm as e:
                folder_exists = False
                print(f"FTP folder does not exist: {ftp_folder_file}")

            if folder_exists:
                # Ensure the destination folder exists
                if not os.path.exists(local_folder_file):
                    os.makedirs(local_folder_file)

                # Download the files from the FTP server to the local folder
                filenames = ftp.nlst()
                for filename in filenames:
                    local_folder_path = r"C:\Users\aslee\OneDrive - MORE ELECTRIC AND POWER CORPORATION\Desktop\DASHBOARD_FINAL\MORE Trading Node"
                    try:
                        with open(local_folder_path, 'wb') as local_file:
                          ftp.retrbinary('RETR ' + filename, local_file.write)
                        print(f"Copied {filename} to {local_folder_path}")
                        # After copy to local filepath (paayos nalang indentation if ever di sakto)
                        match = re.search(r"MPI_LMP_DIPC_(\d{8})(\d{2})(\d{2})", filename)
                        if match:
                            date = match.group(1)
                            time_hh = int(match.group(2))
                            time_ss = int(match.group(3))

                        if (time_ss == 0):
                            if (time_hh == 0):
                              sorted_directory = os.path.join(local_folder_path, date, "23")
                            else:
                              sorted_directory = os.path.join(local_folder_path, date, str(time_hh-1))
                        else:
                            sorted_directory = os.path.join(local_folder_path, date, str(time_hh))
                        print(f"DEBUG sorted directory: {sorted_directory}")
                        os.makedirs(sorted_directory, exist_ok=True)
                        # Move the file to the sorted directory
                        shutil.move(filename, sorted_directory)
                    except ftplib.error_perm as e:
                        print(f"Could not copy {filename}: {e}")

In [6]:
# Main loop to check for new files every 5 minutes
while True:
    download_files()
    time.sleep(300)  # Sleep for 5 minutes

Checking FTP folder path: /LMP/DIPC/20240708
Checking local folder path: C:\Users\aslee\OneDrive - MORE ELECTRIC AND POWER CORPORATION\Desktop\DASHBOARD_FINAL\MORE Trading Node\20240708


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\aslee\\OneDrive - MORE ELECTRIC AND POWER CORPORATION\\Desktop\\DASHBOARD_FINAL\\MORE Trading Node'